In [1]:
# import the necessary libraries
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset

# import dataloader class
from torch.utils.data import DataLoader

from torchvision import transforms

# import visualisation tool
import matplotlib.pyplot as plt

# import time to time the training
import time
